# Qdrant Client - Современное руководство

Полное руководство по работе с qdrant-client v1.12+:
- Работа с коллекциями
- Пополнение коллекций чанками
- Тонкая настройка поиска

## 1. Настройка и импорты

In [ ]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import (
    Distance,
    VectorParams,
    PointStruct,
    Filter,
    FieldCondition,
    MatchValue,
    MatchAny,
    Range,
    SearchParams,
    OptimizersConfigDiff,
    HnswConfigDiff,
    QueryRequest,
)
import numpy as np
from typing import List, Dict, Any

In [ ]:
# Константы
VECTOR_SIZE = 384  # Типичный размер для sentence-transformers
COLLECTION_NAME = "documents"

# Создание клиента (in-memory для демонстрации)
client = QdrantClient(":memory:")

# Для продакшена:
# client = QdrantClient(url="http://localhost:6333")
# client = QdrantClient(url="http://localhost:6333", api_key="your-api-key")

## 2. Работа с коллекциями

### 2.1 Проверка и создание коллекции

In [ ]:
# Проверить существование коллекции
if client.collection_exists(COLLECTION_NAME):
    print(f"Коллекция '{COLLECTION_NAME}' существует")
    client.delete_collection(COLLECTION_NAME)
    print(f"Коллекция удалена для пересоздания")
else:
    print(f"Коллекция '{COLLECTION_NAME}' не существует")

In [ ]:
# Создание коллекции с расширенными настройками
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=VECTOR_SIZE,
        distance=Distance.COSINE,  # Также: EUCLID, DOT, MANHATTAN
    ),
    # Настройки оптимизатора
    optimizers_config=OptimizersConfigDiff(
        indexing_threshold=20000,  # Порог для начала индексации
        memmap_threshold=50000,    # Порог для переноса на диск
    ),
    # Настройки HNSW индекса
    hnsw_config=HnswConfigDiff(
        m=16,              # Количество связей на точку
        ef_construct=100,  # Параметр построения индекса
    ),
)
print(f"Коллекция '{COLLECTION_NAME}' создана")

### 2.2 Информация о коллекции

In [ ]:
# Список всех коллекций
collections = client.get_collections()
print("Все коллекции:")
for col in collections.collections:
    print(f"  - {col.name}")

In [ ]:
# Детальная информация о коллекции
info = client.get_collection(COLLECTION_NAME)
print(f"Статус: {info.status}")
print(f"Количество точек: {info.points_count}")
print(f"Количество сегментов: {info.segments_count}")
print(f"\nКонфигурация векторов:")
print(f"  Размерность: {info.config.params.vectors.size}")
print(f"  Метрика: {info.config.params.vectors.distance}")
print(f"\nHNSW конфигурация:")
print(f"  m: {info.config.hnsw_config.m}")
print(f"  ef_construct: {info.config.hnsw_config.ef_construct}")

### 2.3 Обновление параметров коллекции

In [ ]:
# Обновление параметров оптимизатора
client.update_collection(
    collection_name=COLLECTION_NAME,
    optimizers_config=OptimizersConfigDiff(
        indexing_threshold=10000,
    ),
)
print("Параметры коллекции обновлены")

## 3. Пополнение коллекций данными

### 3.1 Генерация тестовых данных

In [ ]:
# Симуляция текстовых документов с метаданными
np.random.seed(42)

categories = ["technology", "science", "business", "health"]
sources = ["article", "blog", "paper", "news"]

def generate_test_data(n_points: int) -> List[Dict[str, Any]]:
    """Генерация тестовых данных с векторами и payload."""
    data = []
    for i in range(n_points):
        data.append({
            "id": i,
            "vector": np.random.rand(VECTOR_SIZE).tolist(),
            "payload": {
                "text": f"Document {i} content about {categories[i % len(categories)]}",
                "category": categories[i % len(categories)],
                "source": sources[i % len(sources)],
                "page": i % 100,
                "score": round(np.random.uniform(0.5, 1.0), 2),
                "tags": [categories[i % len(categories)], f"tag_{i % 5}"],
            }
        })
    return data

# Генерируем 1000 тестовых документов
test_data = generate_test_data(1000)
print(f"Сгенерировано {len(test_data)} документов")
print(f"\nПример документа:")
print(f"  ID: {test_data[0]['id']}")
print(f"  Payload: {test_data[0]['payload']}")

### 3.2 Простой upsert

In [ ]:
# Добавление нескольких точек через upsert
small_batch = test_data[:10]

client.upsert(
    collection_name=COLLECTION_NAME,
    points=[
        PointStruct(
            id=item["id"],
            vector=item["vector"],
            payload=item["payload"],
        )
        for item in small_batch
    ],
    wait=True,  # Ждать завершения операции
)

info = client.get_collection(COLLECTION_NAME)
print(f"Добавлено точек: {info.points_count}")

### 3.3 Batch upsert с чанками

In [ ]:
def batch_upsert(data: List[Dict], batch_size: int = 100):
    """Загрузка данных батчами."""
    total = len(data)
    for i in range(0, total, batch_size):
        batch = data[i:i + batch_size]
        points = [
            PointStruct(
                id=item["id"],
                vector=item["vector"],
                payload=item["payload"],
            )
            for item in batch
        ]
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=points,
            wait=True,
        )
        print(f"Загружено: {min(i + batch_size, total)}/{total}")

# Загружаем оставшиеся данные (с 10 по 1000)
remaining_data = test_data[10:]
batch_upsert(remaining_data, batch_size=200)

info = client.get_collection(COLLECTION_NAME)
print(f"\nВсего точек в коллекции: {info.points_count}")

### 3.4 Upload points (оптимизированный метод)

In [ ]:
# Создадим вторую коллекцию для демонстрации upload_points
COLLECTION_2 = "documents_v2"

if client.collection_exists(COLLECTION_2):
    client.delete_collection(COLLECTION_2)

client.create_collection(
    collection_name=COLLECTION_2,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE),
)

# upload_points - оптимизированный метод для больших объёмов
# Автоматически разбивает на батчи и поддерживает параллельную загрузку
points = [
    PointStruct(
        id=item["id"],
        vector=item["vector"],
        payload=item["payload"],
    )
    for item in test_data
]

client.upload_points(
    collection_name=COLLECTION_2,
    points=points,
    batch_size=256,    # Размер батча
    parallel=2,        # Количество параллельных процессов
    max_retries=3,     # Повторы при ошибках
)

info = client.get_collection(COLLECTION_2)
print(f"Загружено через upload_points: {info.points_count} точек")

## 4. Поиск и запросы

### 4.1 Базовый поиск

In [ ]:
# Генерируем query вектор
query_vector = np.random.rand(VECTOR_SIZE).tolist()

# Базовый поиск с query_points (современный API)
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    limit=5,
)

print("Топ-5 результатов:")
for point in results.points:
    print(f"  ID: {point.id}, Score: {point.score:.4f}, Category: {point.payload['category']}")

### 4.2 Поиск с фильтрами

In [ ]:
# Поиск с фильтром по категории (must)
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="category",
                match=MatchValue(value="technology"),
            )
        ]
    ),
    limit=5,
)

print("Результаты с фильтром category='technology':")
for point in results.points:
    print(f"  ID: {point.id}, Score: {point.score:.4f}, Category: {point.payload['category']}")

In [ ]:
# Поиск с фильтром по диапазону
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="page",
                range=Range(gte=10, lte=50),  # Страницы от 10 до 50
            ),
            FieldCondition(
                key="score",
                range=Range(gte=0.7),  # Только с высоким score
            ),
        ]
    ),
    limit=5,
)

print("Результаты с фильтром по диапазону (page: 10-50, score >= 0.7):")
for point in results.points:
    print(f"  ID: {point.id}, Page: {point.payload['page']}, Score field: {point.payload['score']}")

In [ ]:
# Комплексный фильтр: must + should + must_not
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="source",
                match=MatchAny(any=["article", "paper"]),  # Статьи или научные работы
            )
        ],
        should=[
            FieldCondition(
                key="category",
                match=MatchValue(value="science"),
            ),
            FieldCondition(
                key="category",
                match=MatchValue(value="technology"),
            ),
        ],
        must_not=[
            FieldCondition(
                key="page",
                range=Range(lt=5),  # Исключить первые 5 страниц
            )
        ],
    ),
    limit=5,
)

print("Комплексный фильтр (source in [article, paper], category: science/technology, page >= 5):")
for point in results.points:
    print(f"  ID: {point.id}, Source: {point.payload['source']}, Category: {point.payload['category']}")

### 4.3 Тонкая настройка параметров поиска

In [ ]:
# Расширенные параметры поиска
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    # Параметры качества поиска
    search_params=SearchParams(
        hnsw_ef=128,    # Больше = точнее, но медленнее (по умолчанию: ef_construct)
        exact=False,    # True = точный поиск без индекса (медленно)
    ),
    # Минимальный порог релевантности
    score_threshold=0.5,
    # Выбор полей в ответе
    with_payload=["text", "category"],  # Только указанные поля
    with_vectors=False,  # Не возвращать векторы
    limit=5,
)

print("Поиск с расширенными параметрами:")
for point in results.points:
    print(f"  ID: {point.id}, Score: {point.score:.4f}")
    print(f"    Payload: {point.payload}")

In [ ]:
# Пагинация через limit + offset
page = 2
page_size = 5

results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=query_vector,
    limit=page_size,
    offset=(page - 1) * page_size,  # Пропустить первую страницу
)

print(f"Страница {page} (offset={(page - 1) * page_size}):")
for point in results.points:
    print(f"  ID: {point.id}, Score: {point.score:.4f}")

### 4.4 Batch-поиск (несколько запросов)

In [ ]:
# Несколько поисковых запросов за один вызов
query_vectors = [
    np.random.rand(VECTOR_SIZE).tolist(),
    np.random.rand(VECTOR_SIZE).tolist(),
    np.random.rand(VECTOR_SIZE).tolist(),
]

# Общий фильтр для всех запросов
common_filter = Filter(
    must=[
        FieldCondition(
            key="category",
            match=MatchValue(value="technology"),
        )
    ]
)

# Batch запросы
batch_results = client.query_batch_points(
    collection_name=COLLECTION_NAME,
    requests=[
        QueryRequest(
            query=qv,
            filter=common_filter,
            limit=3,
            with_payload=["category", "source"],
        )
        for qv in query_vectors
    ],
)

print(f"Batch-поиск: {len(batch_results)} запросов")
for i, result in enumerate(batch_results):
    print(f"\nЗапрос {i + 1}:")
    for point in result.points:
        print(f"  ID: {point.id}, Score: {point.score:.4f}")

## 5. Дополнительные операции

### 5.1 Получение точек по ID

In [ ]:
# Получить конкретные точки по ID
points = client.retrieve(
    collection_name=COLLECTION_NAME,
    ids=[0, 5, 10],
    with_payload=True,
    with_vectors=False,
)

print("Точки по ID [0, 5, 10]:")
for point in points:
    print(f"  ID: {point.id}, Category: {point.payload['category']}")

### 5.2 Scroll (итерация по всем точкам)

In [ ]:
# Scroll - получение всех точек с пагинацией
scroll_result, next_offset = client.scroll(
    collection_name=COLLECTION_NAME,
    scroll_filter=Filter(
        must=[
            FieldCondition(
                key="category",
                match=MatchValue(value="science"),
            )
        ]
    ),
    limit=10,
    with_payload=["category", "page"],
    with_vectors=False,
)

print(f"Scroll результаты (первые 10 из category='science'):")
for point in scroll_result:
    print(f"  ID: {point.id}, Page: {point.payload['page']}")
print(f"\nNext offset: {next_offset}")

In [ ]:
# Полная итерация через scroll
def scroll_all(collection_name: str, filter_obj=None, batch_size: int = 100):
    """Итерация по всем точкам коллекции."""
    offset = None
    all_points = []
    
    while True:
        points, next_offset = client.scroll(
            collection_name=collection_name,
            scroll_filter=filter_obj,
            limit=batch_size,
            offset=offset,
            with_payload=True,
            with_vectors=False,
        )
        all_points.extend(points)
        
        if next_offset is None:
            break
        offset = next_offset
    
    return all_points

# Получить все точки категории 'health'
health_filter = Filter(
    must=[FieldCondition(key="category", match=MatchValue(value="health"))]
)
health_points = scroll_all(COLLECTION_NAME, health_filter)
print(f"Всего точек в категории 'health': {len(health_points)}")

### 5.3 Подсчёт точек

In [ ]:
# Подсчёт точек с фильтром
count = client.count(
    collection_name=COLLECTION_NAME,
    count_filter=Filter(
        must=[
            FieldCondition(
                key="source",
                match=MatchValue(value="article"),
            )
        ]
    ),
    exact=True,  # Точный подсчёт
)

print(f"Количество точек с source='article': {count.count}")

### 5.4 Обновление payload

In [ ]:
# Обновить payload для конкретных точек
client.set_payload(
    collection_name=COLLECTION_NAME,
    payload={
        "updated": True,
        "priority": "high",
    },
    points=[0, 1, 2],  # ID точек
)

# Проверяем обновление
updated_points = client.retrieve(
    collection_name=COLLECTION_NAME,
    ids=[0, 1, 2],
    with_payload=True,
)

print("Обновлённые точки:")
for point in updated_points:
    print(f"  ID: {point.id}, Updated: {point.payload.get('updated')}, Priority: {point.payload.get('priority')}")

In [ ]:
# Полная замена payload (удаляет старые поля)
client.overwrite_payload(
    collection_name=COLLECTION_NAME,
    payload={
        "new_field": "completely_new",
    },
    points=[999],
)

point = client.retrieve(COLLECTION_NAME, ids=[999], with_payload=True)
print(f"Перезаписанный payload для ID=999: {point[0].payload}")

### 5.5 Удаление точек

In [ ]:
# Удаление по ID
count_before = client.count(collection_name=COLLECTION_NAME, exact=True).count

client.delete(
    collection_name=COLLECTION_NAME,
    points_selector=models.PointIdsList(points=[999, 998, 997]),
)

count_after = client.count(collection_name=COLLECTION_NAME, exact=True).count
print(f"Удалено точек: {count_before - count_after}")

In [ ]:
# Удаление по фильтру
count_before = client.count(collection_name=COLLECTION_NAME, exact=True).count

client.delete(
    collection_name=COLLECTION_NAME,
    points_selector=models.FilterSelector(
        filter=Filter(
            must=[
                FieldCondition(
                    key="page",
                    range=Range(gte=95),  # Удалить страницы >= 95
                )
            ]
        )
    ),
)

count_after = client.count(collection_name=COLLECTION_NAME, exact=True).count
print(f"Удалено по фильтру (page >= 95): {count_before - count_after}")

## 6. Очистка

In [ ]:
# Удаление коллекций
client.delete_collection(COLLECTION_NAME)
client.delete_collection(COLLECTION_2)

print("Коллекции удалены")
print(f"Оставшиеся коллекции: {client.get_collections().collections}")

## Резюме

### Ключевые методы qdrant-client v1.12+:

| Операция | Метод |
|----------|-------|
| Создание коллекции | `create_collection()` |
| Добавление точек | `upsert()`, `upload_points()` |
| Поиск | `query_points()` (современный API) |
| Batch-поиск | `query_batch_points()` |
| Итерация | `scroll()` |
| Получение по ID | `retrieve()` |
| Подсчёт | `count()` |
| Обновление | `set_payload()`, `overwrite_payload()` |
| Удаление | `delete()` |

### Важные параметры поиска:
- `query_filter` - фильтрация по payload
- `search_params` - настройки HNSW (hnsw_ef, exact)
- `score_threshold` - минимальный порог релевантности
- `with_payload` - выбор возвращаемых полей
- `limit`, `offset` - пагинация